In [1]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim

from utils.dataset import FeatureCaptionDataset
from utils.models import CNNEncoder, RNNDecoder
from utils.helpers import get_tokenizer_vocab, collate_fn_pad
from utils.trainer import Trainer

In [2]:
PATH = "dataset/"

In [3]:
df = pd.read_csv(PATH + "captions.txt")

In [4]:
df.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [5]:
tokenizer, vocab = get_tokenizer_vocab(df)

In [6]:
dataset = FeatureCaptionDataset(PATH + "embeddings/", df, tokenizer, vocab)

In [7]:
encoder_dim, decoder_dim, attention_dim, embed_dim, vocab_size, max_len = 512, 256, 128, 64, len(vocab), 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
encoder = CNNEncoder(1536, encoder_dim)

In [9]:
decoder = RNNDecoder(encoder_dim=encoder_dim, 
                     decoder_dim=decoder_dim, 
                     attention_dim=attention_dim, 
                     embedding_dim=embed_dim, 
                     vocab_size=vocab_size
                    )

In [10]:
encoder.load_state_dict(torch.load("models/encoder.pt"))
encoder.eval()

decoder.load_state_dict(torch.load("models/decoder.pt"))
decoder.eval()

RNNDecoder(
  (embedding): Embedding(9213, 64)
  (gru): GRU(576, 256, batch_first=True)
  (fc_1): Linear(in_features=256, out_features=256, bias=True)
  (fc_2): Linear(in_features=256, out_features=9213, bias=True)
  (attn): Attention(
    (encoder_attn): Linear(in_features=512, out_features=128, bias=False)
    (decoder_attn): Linear(in_features=256, out_features=128, bias=False)
    (value): Linear(in_features=128, out_features=1, bias=False)
    (softmax): Softmax(dim=1)
  )
  (softmax): Softmax(dim=1)
)

In [11]:
img, cap = dataset[0]

In [22]:
enc = encoder(torch.tensor(torch.tensor(img).unsqueeze(0)))

<ipython-input-22-ca9c518fe309>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  enc = encoder(torch.tensor(torch.tensor(img).unsqueeze(0)))


In [23]:
dec = decoder(enc)

TypeError: forward() missing 2 required positional arguments: 'encoder_output' and 'hidden'